In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import optuna

# Load data

In [3]:
path = r"C:\Users\matta\Desktop\Documents\Python\Geolocation\climate_data\working_data\clean_labeled_climate_data.pkl"
df = pd.read_pickle(path)
df.shape

(470342, 125)

In [6]:
temp_precip_cols = ['longitude', 'latitude',
                    'jan_precip', 'feb_precip', 'mar_precip', 'apr_precip', 'may_precip', 'jun_precip', 
                    'jul_precip', 'aug_precip', 'sep_precip', 'oct_precip', 'nov_precip', 'dec_precip',  
                    'jan_meant', 'feb_meant', 'mar_meant', 'apr_meant', 'may_meant', 'jun_meant', 
                    'jul_meant', 'aug_meant', 'sep_meant', 'oct_meant', 'nov_meant', 'dec_meant']
select_cols = ['longitude', 'latitude', 'jan_tmin', 'jul_tmax', 'jan_dptmean', 'jul_dptmean', 'annual_precip']
target_cols = ['Level_1', 'Level_2', 'Level_3', 'Level_4', 'ECO_NAME', 'climates_f']